# Python

***
###  

In [1]:
import os
import copy
import itertools
import numpy as np
import pandas as pd
from sklearn.externals import joblib

In [2]:
# Count the frequency of k-mer in each RNA sequence
#k-mer was normalized by total k-mer count of each RNA sequence
def _count_kmer(Dataset,k): # k = 3,4,5,6,7 & length info
    
    # copy dataset
    dataset = copy.deepcopy(Dataset)
    # alphbet of nucleotide
    nucleotide = ['A','C','G','T']
    
    # generate k-mers
    #  k == 5:
    five = list(itertools.product(nucleotide,repeat=5))
    pentamer = []
    for n in five:
        pentamer.append("".join(n))
    
    #  k == 4:
    four = list(itertools.product(nucleotide,repeat=4))
    tetramer = []
    for n in four:
        tetramer.append("".join(n))

    # k == 3:
    three = list(itertools.product(nucleotide,repeat=3))
    threemer = []
    for n in three:
        threemer.append("".join(n))
    
    # input features can be combinations of diffrent k values
    if k == 34:
        table_kmer = dict.fromkeys(threemer,0)
        table_kmer.update(dict.fromkeys(tetramer,0))
    if k == 45:
        table_kmer = dict.fromkeys(tetramer,0)
        table_kmer.update(dict.fromkeys(pentamer,0))
    if k == 345:
        table_kmer = dict.fromkeys(threemer,0)
        table_kmer.update(dict.fromkeys(tetramer,0))
        table_kmer.update(dict.fromkeys(pentamer,0))

    # count k-mer for each sequence
    for mer in table_kmer.keys():
        table_kmer[mer] = dataset["cdna"].apply(lambda x : x.count(mer))
    
    # for k-mer raw count without normalization, index: nuc:1 or cyto:0
    rawcount_kmer_df = pd.DataFrame(table_kmer)
    df1_rawcount = pd.concat([rawcount_kmer_df,dataset["ensembl_transcript_id"]],axis = 1)
    #df1_rawcount.index = dataset["tag"]

    # for k-mer frequency with normalization , index: nuc:1 or cyto:0
    freq_kmer_df = rawcount_kmer_df.apply(lambda x: x/x.sum(),axis=1)
    df1 = pd.concat([freq_kmer_df,dataset["ensembl_transcript_id"]],axis = 1)
    #df1.index = dataset["tag"]

    return df1,df1_rawcount

In [19]:
# Input dir 
RNALight_lncRNA_input_dir = "../../lncRNA/03_Model_Construction/01_Machine_Learning_Model/01_ML_Model_Output"
RNALight_mRNA_input_dir = "../../mRNA/03_Model_Construction/01_Machine_Learning_Model/01_ML_Model_Output"

# Load model
RNALight_lncRNA = joblib.load(os.path.join(RNALight_lncRNA_input_dir,"LightGBM/best_LightGBM_model.pkl"))
RNALight_mRNA = joblib.load(os.path.join(RNALight_mRNA_input_dir,"LightGBM/best_LightGBM_model.pkl"))

***
###   

## 1. lncRNA with intron

In [4]:
# Load lncRNA with intron file
lncRNA_with_intron_file = pd.read_csv("./01_RNA_retaining_intron_files/gencode.v30.lncRNA_transcripts_with_intron_major_compact_trans_id.tsv",sep = '\t',header= None)
lncRNA_with_intron_file.columns = ["ensembl_transcript_id","name","cdna"]

In [5]:
df_kmer_345,df_kmer_345_rawcount = _count_kmer(lncRNA_with_intron_file,345)
df_kmer_345.to_csv("lncRNA_with_intron_df_kmer345_freq.tsv",sep='\t')
df_kmer_345_rawcount.to_csv("lncRNA_with_intron_df_kmer345_rawcount.tsv",sep='\t')

del df_kmer_345['ensembl_transcript_id']
x_kmer = df_kmer_345.values

y_pred = RNALight_lncRNA.predict(x_kmer)
y_prob = RNALight_lncRNA.predict_proba(x_kmer)[:,1]

In [6]:
lncRNA_with_intron_file["RNALight_pred_label"] =  y_pred
lncRNA_with_intron_file["RNALight_pred_prob"] = y_prob
lncRNA_with_intron_file.to_csv("lncRNA_with_intron_file_predict_df.tsv",sep = '\t',index=False)

***
###   

## 2. mRNA with intron

In [20]:
# Load mRNA with intron file
mRNA_with_intron_file = pd.read_csv("./01_RNA_retaining_intron_files/gencode.v30.pc_mRNA_transcripts_with_intron_major_compact_trans_id.tsv",sep = '\t',header= None)
mRNA_with_intron_file.columns = ["ensembl_transcript_id","name","cdna"]

In [21]:
df_kmer_345,df_kmer_345_rawcount = _count_kmer(mRNA_with_intron_file,345)
df_kmer_345.to_csv("mRNA_with_intron_df_kmer345_freq.tsv",sep='\t')
df_kmer_345_rawcount.to_csv("mRNA_with_intron_df_kmer345_rawcount.tsv",sep='\t')

del df_kmer_345['ensembl_transcript_id']
x_kmer = df_kmer_345.values

y_pred = RNALight_mRNA.predict(x_kmer)
y_prob = RNALight_mRNA.predict_proba(x_kmer)[:,1]

In [22]:
mRNA_with_intron_file["RNALight_pred_label"] =  y_pred
mRNA_with_intron_file["RNALight_pred_prob"] = y_prob
mRNA_with_intron_file.to_csv("mRNA_with_intron_file_predict_df.tsv",sep = '\t',index=False)

***
###   

## 2. circRNA with intron

In [15]:
# Load circRNA with intron file
circRNA_with_intron_file = pd.read_csv("./01_RNA_retaining_intron_files/PA1_high_exp_circ_info_with_intron_seq.tsv",sep = '\t')
circRNA_with_intron = circRNA_with_intron_file.iloc[:,[4,4,8]]
circRNA_with_intron.columns = ["ensembl_transcript_id","name","cdna"]

In [16]:
df_kmer_345,df_kmer_345_rawcount = _count_kmer(circRNA_with_intron,345)
df_kmer_345.to_csv("circRNA_with_intron_df_kmer345_freq.tsv",sep='\t')
df_kmer_345_rawcount.to_csv("circRNA_with_intron_df_kmer345_rawcount.tsv",sep='\t')

del df_kmer_345['ensembl_transcript_id']
x_kmer = df_kmer_345.values

y_pred = RNALight_lncRNA.predict(x_kmer)
y_prob = RNALight_lncRNA.predict_proba(x_kmer)[:,1]

In [17]:
circRNA_with_intron["RNALight_pred_label"] =  y_pred
circRNA_with_intron["RNALight_pred_prob"] = y_prob
circRNA_with_intron.to_csv("circRNA_with_intron_file_predict_df.tsv",sep = '\t',index=False)

/picb/rnomics3/yuanguohua/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/picb/rnomics3/yuanguohua/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
